# Отзывы на экскурсии Tripster.ru — обработка текста

In [1]:
import warnings
warnings.simplefilter('ignore')

import os
import re
import functools
from collections import Counter, defaultdict
from operator import itemgetter
import pandas as pd
import numpy as np
import nltk

data_dir = "./data"

## Загрузка данных

In [3]:
def data_path(relative_path):
    return os.path.join(data_dir, relative_path)

def load_reviews():
    print('Loading reviews...')
    reviews = pd.read_csv(data_path('reviews_filtered.csv'))
    print('Total: {}'.format(len(reviews)))
    return reviews

In [4]:
reviews = load_reviews()
reviews.head()

Loading reviews...
Total: 32973


,id,subject,author,rate,text,text_len
0,2,1381,1024,5.0,"Экскурсия прошла просто отлично, все понравило...",133
1,3,1341,1040,5.0,У вас очень удобный и полезный сервис! Был при...,594
2,4,1341,689,5.0,Хотела оставить слова благодарности Александру...,811
3,6,1379,1053,5.0,В качестве отзыва об экскурсиях - все только в...,825
4,7,1533,640,NaN,"Мы заранее договорились об экскурсии с Анной, ...",1126


## Анализ и очистка символов

### Анализ

In [5]:
all_texts = ' '.join(reviews.text).lower()

In [6]:
print("Count of unique symbols: {}".format(len(set(all_texts))))

Count of unique symbols: 199


In [7]:
chars_counts = Counter(all_texts)

In [8]:
char_groups = {
    'ru': re.compile(r'[а-яё]'),
    'lat': re.compile(r'[a-z]'),
    'punctuation': re.compile(r'[\.,\!\?"«»“”„()\-–—\:;\'…]'),    
    'space': re.compile(r'[\s]'),    
    'num': re.compile(r'[\d]'),    
    'url special chars': re.compile(r'[/&=#_+%]'),    
}

chars_count_grouped = defaultdict(int)
for char, count in chars_counts.items():
    char_group = char
    for group_name, regexp in char_groups.items():
        if regexp.match(char):
            char_group = group_name
            continue
    
    chars_count_grouped[char_group] += count

In [9]:
for char_group, count in sorted(chars_count_grouped.items(), key=itemgetter(1), reverse=True):
    print ('{}: {}'.format(char_group, count))

ru: 10127389
space: 1819319
punctuation: 415849
lat: 40682
num: 24116
url special chars: 2733
і: 147
­: 64
י: 31
є: 30
️: 28
*: 27
ו: 27
№: 25
’: 22
！: 22
ל: 20
ї: 17
ר: 17
•: 16
á: 15
מ: 14
: 13
\: 12
נ: 12
ה: 12
ם: 12
ב: 11
א: 10
​: 9
ü: 8
~: 8
€: 7
₽: 7
ת: 7
[: 6
<: 6
@: 6
‎: 6
ד: 6
ש: 6
ט: 6
]: 5
é: 5
$: 5
>: 5
°: 5
☺: 5
|: 5
צ: 5
ş: 4
ö: 4
́: 4
）: 4
ג: 4
ח: 4
כ: 4
^: 3
`: 3
ç: 3
（: 3
ä: 3
ã: 3
ז: 3
פ: 3
ק: 3
ע: 3
‌: 3
‘: 3
œ: 2
¬: 2
ß: 2
ο: 2
υ: 2
ס: 2
â: 1
´: 1
à: 1
κ: 1
τ: 1
σ: 1
π: 1
ε: 1
ι: 1
ά: 1
: 1
º: 1
ñ: 1
ך: 1
ץ: 1
ף: 1
，: 1
ë: 1
あ: 1
り: 1
が: 1
と: 1
ご: 1
ざ: 1
い: 1
ま: 1
す: 1
よ: 1
。: 1
♡: 1


### Очистка

In [10]:
RE_WORDS = re.compile(r"[а-яё\-a-z]+", re.UNICODE | re.IGNORECASE)

def filter_chars(text, regex=RE_WORDS):
    text = text.replace('т.е.', '')
    return " ".join(regex.findall(text)).replace(' - ', ' ')

In [11]:
all_texts_words = filter_chars(all_texts)

In [12]:
all_texts_words[:3000]

'экскурсия прошла просто отлично все понравилось было очень увлекательно и интересно узнали много нового о ватикане и риме в целом у вас очень удобный и полезный сервис был приятно удивлен что за сравнительно небольшие деньги можно получить такой выбор экскурсий которые помогут лучше познакомиться с городами наш гид александр был очень отзывчивым и приветливым молодым человеком он хорошо владел информацией о риме а также с ним было приятно общаться и на другие темы мы получили от него полезные советы где лучше кушать и делать покупки заказав эту услугу вы лучше познакомитесь с городом сэкономите время и деньги и вам будет к кому обратиться за консультацией если что-то окажется не понятно в чужом городе хотела оставить слова благодарности александру за неоценимую помощь во время путешествия в рим в феврале наши римские каникулы в марте года удались мы заранее оговорили наши тематические туры по городу спасибо саше за четкость и быстроту в переписке он действительно настоящий профессиона

## Анализ и очистка слов

### Считаем слова, ищем стоп-слова

In [13]:
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('russian') + stopwords.words('english'))

def filter_stop_words(words, stop_words):
    return list(filter(lambda word: word not in stop_words, words))

In [14]:
all_words_filtered = filter_stop_words(all_texts_words.split(' '), STOPWORDS)

In [15]:
def most_common_words(words, count=30):
    word_counts = Counter(words)
    return pd.DataFrame(word_counts.most_common(count), columns=['word', 'count'])

In [16]:
most_common_words(all_words_filtered, 30)

,word,count
0,очень,28267
1,спасибо,19464
2,экскурсия,16295
3,экскурсию,11533
4,экскурсии,10886
5,это,9428
6,нам,8921
7,интересно,6729
8,всем,6266
9,время,5947


In [17]:
STOPWORDS = STOPWORDS.union({'это', 'которые', 'очень', 'просто', 'http', 'experience', 'tripster', 'ru'})
all_words_filtered = filter_stop_words(all_words_filtered, STOPWORDS)
most_common_words(all_words_filtered)

,word,count
0,спасибо,19464
1,экскурсия,16295
2,экскурсию,11533
3,экскурсии,10886
4,нам,8921
5,интересно,6729
6,всем,6266
7,время,5947
8,большое,5545
9,гид,5539


In [18]:
' '.join(all_words_filtered)[:3000]

'экскурсия прошла отлично понравилось увлекательно интересно узнали нового ватикане риме целом удобный полезный сервис приятно удивлен сравнительно небольшие деньги получить выбор экскурсий помогут познакомиться городами наш гид александр отзывчивым приветливым молодым человеком владел информацией риме также приятно общаться другие темы получили полезные советы кушать делать покупки заказав услугу познакомитесь городом сэкономите время деньги кому обратиться консультацией что-то окажется понятно чужом городе хотела оставить слова благодарности александру неоценимую помощь время путешествия рим феврале наши римские каникулы марте года удались заранее оговорили наши тематические туры городу спасибо саше четкость быстроту переписке действительно настоящий профессионал отлично знающий рим итальянский язык культуру чувствовали александр старается именно ощущали какими-то тысячными туристами всё делается шаблону подарил нам настоящий праздник высоком профессиональном уровне души рекомендую в

In [19]:
def build_stop_words():
    return set(
        stopwords.words('russian') + 
        stopwords.words('english') +
        ['это', 'которые', 'очень', 'просто', 'http', 'experience', 'tripster', 'ru']
    )

## Разбиваем на предложения

In [49]:
sent_tokenize = nltk.data.load('russian.pickle')

def split_sentenses(text):
    return sent_tokenize.tokenize(text)

In [50]:
split_sentenses(reviews.text[1])

['У вас очень удобный и полезный сервис!',
 'Был приятно удивлен, что за сравнительно небольшие деньги можно получить такой выбор экскурсий, которые помогут лучше познакомиться с городами.',
 'Наш гид Александр, был очень отзывчивым и приветливым молодым человеком.',
 'Он хорошо владел информацией о Риме, а также с ним было приятно общаться и на другие темы.',
 'Мы получили от него полезные советы где лучше кушать и делать покупки.',
 'Заказав эту услугу, вы лучше познакомитесь с городом, сэкономите время и деньги, и вам будет к кому обратиться за консультацией, если что-то окажется не понятно в чужом городе.']

## Объединяем всё вместе

In [51]:
RE_WORDS = re.compile(r"[а-яё\-a-z]+", re.UNICODE | re.IGNORECASE)

# Двойной разрыв строки считаем эквивалентом разбиения на предложения
def double_linebreak_to_dot(text):
    return text.replace('\r\n\r\n', '. ')

FILTER_MINUS = lambda word: word != '-'

# Оставляет только слова на русском и литинице, удаляя всё лишнее
# возвращает итератор
def split_words(text, regex=RE_WORDS):
    return filter(FILTER_MINUS, regex.findall(text))

# Генерирует список стоп-слов для русского и английского
def build_stop_words():
    return set(
        stopwords.words('russian') + 
        stopwords.words('english') +
        ['это', 'которые', 'очень', 'просто', 'http', 'experience', 'tripster', 'ru']
    )

STOPWORDS = build_stop_words()

# Фильтрует стоп-слова, возвращает итератор
def filter_stop_words(words, stop_words=STOPWORDS):
    return filter(lambda word: word not in stop_words and len(word) > 1, words)

sent_tokenize = nltk.data.load('russian.pickle')

# Разбивает текст на предложения
# Возвращает список
def split_sentenses(text):
    text = double_linebreak_to_dot(text)
    return sent_tokenize.tokenize(text)

def join_words(words):
    return ' '.join(words)

### Очистка текста без учёта предложений

In [23]:
def clean_text(text):
    text = text.lower()
    words = split_words(text)
    words = filter_stop_words(words)
    return join_words(words).strip()

In [24]:
reviews.text[1]

'У вас очень удобный и полезный сервис! Был приятно удивлен, что за сравнительно небольшие деньги можно получить такой выбор экскурсий, которые помогут лучше познакомиться с городами. Наш гид Александр, был очень отзывчивым и приветливым молодым человеком. Он хорошо владел информацией о Риме, а также с ним было приятно общаться и на другие темы. Мы получили от него полезные советы где лучше кушать и делать покупки. Заказав эту услугу, вы лучше познакомитесь с городом, сэкономите время и деньги, и вам будет к кому обратиться за консультацией, если что-то окажется не понятно в чужом городе.'

In [25]:
clean_text(reviews.text[1])

'удобный полезный сервис приятно удивлен сравнительно небольшие деньги получить выбор экскурсий помогут познакомиться городами наш гид александр отзывчивым приветливым молодым человеком владел информацией риме также приятно общаться другие темы получили полезные советы кушать делать покупки заказав услугу познакомитесь городом сэкономите время деньги кому обратиться консультацией что-то окажется понятно чужом городе'

### Сохраняем отзывы с очищенным ткстом

In [26]:
# Обрабатывает текст отзывов с помощью text_processor и сохраняет 
def process_text_and_save(reviews, file_name, text_processor):
    reviews = reviews.copy()
    reviews['text'] = reviews.text.map(text_processor)
    reviews['text_len'] = reviews.text.str.len()
    reviews = reviews.loc[reviews.text.str.len() > 1]
    reviews.to_csv(data_path(file_name + '.csv'), index=False)
    return reviews

In [27]:
reviews_text_cleaned = process_text_and_save(reviews, 'reviews_text_cleaned', clean_text)
reviews_text_cleaned.text[1]

'удобный полезный сервис приятно удивлен сравнительно небольшие деньги получить выбор экскурсий помогут познакомиться городами наш гид александр отзывчивым приветливым молодым человеком владел информацией риме также приятно общаться другие темы получили полезные советы кушать делать покупки заказав услугу познакомитесь городом сэкономите время деньги кому обратиться консультацией что-то окажется понятно чужом городе'

### Очистка текста с предварительным разбиением на предложения

In [52]:
def clean_sentences(text):
    sentences = split_sentenses(text)
    return filter(len, map(clean_text, sentences))

In [53]:
'|'.join(clean_sentences(reviews.text[1]))

'удобный полезный сервис|приятно удивлен сравнительно небольшие деньги получить выбор экскурсий помогут познакомиться городами|наш гид александр отзывчивым приветливым молодым человеком|владел информацией риме также приятно общаться другие темы|получили полезные советы кушать делать покупки|заказав услугу познакомитесь городом сэкономите время деньги кому обратиться консультацией что-то окажется понятно чужом городе'

In [30]:
reviews.columns

Index(['id', 'subject', 'author', 'rate', 'text', 'text_len'], dtype='object')

In [55]:
# Разбивает текст отзывов на предложения с помощью  sentence_processor и сохраняет 
def process_sentences_and_save(reviews, file_name, sentence_processor):
    reviews = pd.DataFrame([
        (
            review.id, 
            sentence_num,
            review.subject, 
            review.author, 
            review.rate, 
            clean_text(sentence),
            sentence,
        )
        for review_num, review in reviews.iterrows()       
        for sentence_num, sentence in enumerate(sentence_processor(review.text), 1)
    ], columns=[
        'id', 'sentence_num', 'subject', 'author', 'rate', 'text', 'text_orig'
    ])
    reviews['text_len'] = reviews.text.str.len()
    reviews = reviews.loc[reviews.text.str.len() > 1]    
    reviews.to_csv(data_path(file_name + '.csv'), index=False)
    return reviews

In [57]:
%%time
reviews_sentences_cleaned = process_sentences_and_save(
    reviews, 'reviews_sentences_cleaned', split_sentenses
)
reviews_sentences_cleaned.text[1]

CPU times: user 20.4 s, sys: 148 ms, total: 20.5 s
Wall time: 20.6 s


## Лемматизация

In [39]:
from pymystem3 import Mystem

mystem = Mystem()

def lemmatize(text):
    lemmas = mystem.lemmatize(text)
    return ''.join(lemmas).strip()

In [40]:
lemmatize(reviews_text_cleaned.text[1])

'удобный полезный сервис приятно удивлять сравнительно небольшой деньги получать выбор экскурсия помогать познакомиться город наш гид александр отзывчивый приветливый молодой человек владеть информация рим также приятно общаться другой тема получать полезный совет кушать делать покупка заказывать услуга познакомиться город сэкономить время деньги кто обращаться консультация что-то оказываться понятно чужой город'

### Лемматизация и сохранение отзывов с очищенным текстом

In [45]:
def save_lemmatized(reviews, file_name, lemmatize):
    reviews = reviews.copy()
    reviews['text'] = reviews.text.map(lemmatize)
    reviews['text_len'] = reviews.text.str.len()
    reviews.to_csv(data_path(file_name + '.csv'), index=False)
    return reviews

In [44]:
%%time
reviews_text_lemmatized = save_lemmatized(
    reviews_text_cleaned, 
    'reviews_text_lemmatized',
    lemmatize
)

CPU times: user 5.84 s, sys: 542 ms, total: 6.39 s
Wall time: 51.7 s


In [58]:
%%time
reviews_sentences_lemmatized = save_lemmatized(
    reviews_sentences_cleaned, 
    'reviews_sentences_lemmatized',
    lemmatize
)

CPU times: user 13.1 s, sys: 1.98 s, total: 15.1 s
Wall time: 1min 5s
